<a href="https://colab.research.google.com/github/abdulbaseet-zahir/Kurdish-next-word-predictor/blob/main/Notebooks/kurdish_next_word_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import keras
import tensorflow

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding

from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

from string import punctuation
import os
import re
import pickle
import collections
import numpy as np
import time
import datetime

In [3]:
dataset_file = "/content/drive/MyDrive/kurdish next word prediction/AsoSoft mini-Text Corpus- Version 1.0 (2018-12-10).txt"

In [6]:
def open_clean(dataset_file):
    # output: open and clean text file then return all text as one string

    with open(dataset_file, "r", encoding = "utf8") as f:
        articles = [line for line in f]

    
    raw_data = ''
    raw_data = ' '. join(articles)
    
    data = raw_data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
    data = re.sub(r'[^\w\s]','', data)

    return data

In [55]:
data = open_clean(dataset_file)[:10000]

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer, open('/content/drive/MyDrive/kurdish next word prediction/bigtokenizer.pkl', 'wb'))

In [57]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [58]:
vocab_size = len(tokenizer.word_index) + 1

sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)

sequences = np.array(sequences)

X1 = []
y1 = []

for i in sequences:
    X1.append(i[0])
    y1.append(i[1])
    
X = np.array(X1)
Y = np.array(y1)

In [59]:
len(X)

1398

In [52]:
def data_gen(ins, outs, vocab_size, batch_size=100):
    
    while True:
        x = ins[i:batch_size]
        y = to_categorical(outs[i:batch_size], num_classes=vocab_size)
        
        batch_size=100+batch_size
        yield x, y


In [47]:
model = tensorflow.keras.models.Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

current_time = str(int(time.time()))
current_time2 = '_'.join(str(time.asctime()).split())
current_time3 = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint("nextword.h5", monitor='accuracy', verbose=1, save_best_only=True, mode='auto')
reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)
tensorboard_Visualization = TensorBoard('logs\\'+current_time3, histogram_freq=1)

from tensorflow.keras.optimizers import Adam
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001), metrics=['accuracy'])


In [ ]:
g = data_gen(X,Y,vocab_size)
model.fit_generator(g, 10, epochs=150 , callbacks=[checkpoint, reduce, tensorboard_Visualization])

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

string_word = 'کەناکرێت هەروا بەئاسانی تێیان بڕوانین بەڵکو بۆبکرێت خەڵکی شیاویان بۆبدۆزرێتەوە بەلام وەخت لەمیزاج تەکەتول دووربکەوینەوە هەنگاونان لێرەدا ئەوپەڕی دیموکراتیەت ئازادی بەخۆوە ببینێت کەسی شیاو بەدەنگی بچێتە جێیە بەتالانە ئەوەش لەبەرچاو بگرین دەیان سەرکردەی شەهیدبوون چەندین جار ئەوە هاتووە شوێنی سەرکردەکانی چۆڵ بن دواتر پڕبکرێنەوە لەزۆربەی کاتەکان بەرزکردنەوەی کەسێک یان کەسانێک بۆپڕکردنەوەی شوێنە بەتاڵەکان کراوە بۆیە زۆرجار دەرنەچووە کەچاوەڕوان واتە بەرزکردنەوە کەمتر لە خۆدەگرێت لەوەی هەڵبژاردنی پەیڕەوکردنی سزاو پاداشت گۆڕانە بەخۆیەوە دەبینێت دەستبەکار کەبەشێکی گرنگی کابینەیە پێکدەهێنێ گرێ دەدات بۆرێکخستنەوەی ماڵی یەکێتی لەگشت لایەن کەسەکان ڕووداوە بەگرنگ بخوێنێتەوە پلینۆمیش تەنیا سەرخستن جێ پڕکردنەوەو جێگۆڕکێ نەبێت بۆپەیڕەوکردنی پاداشتیش کەکاری خراپیان کردووە باشیان بگیرێت کەئەو کەسانەی نەیان توانیوە کارەکانیان جێبەجێبکەن بکەن بۆهەڤالانی هاتنەپێشەوە قولایی سیاسییەکان دوێنێ 31 10 2009 سێیەمین دانیشتنی یەکێتیی نیشتمانیی لەتەلاری هونەر لەسلێمانی بەڕێوەچوو تێیدا ڕاپۆرتی لیژنەی ناوچەکانی لەلایەن ڕزگار عەلی خوێنرایەوە ئاماژەیکرد بۆئەو ناوچانە کەپارێزگای کەرکوک سنووری نەینەواو دیالەو سەلاحەددین واست بەغدا دەگرێتەوە مێژووی سیاسەتی مەترسیداری تەعریب تەهجیرو دابڕاندنی ناوچانەی خستەڕوو پەنجەشی خستەسەر کێشەکانی دانیشتووانی بەدەست زوڵمی یاسایی ئەمنی ئابوورییەوە دەناڵێنن ئاسانکاری کێشەکان گوتیشی ماددەی 140 ی دەستوور ماددەیەکی دەستووریی UN گەڕایەوەسەر پارێزگای هەولێر تێبینی ڕەخنەکانی ئەندامان لەپێناوی زیاتر دەوڵەمەندکردنی کەگفتوگۆیەکی هەمەلایەنەی لەبارەوە کراو پەرۆشیی بەرجەستەدەکرد بەگەڕانەوەی جێناکۆک جێبەجێکردنی دەستووری جێناکۆکی لەوبارەیەشەوە مەلا بەختیار ئەندامێکی خانەقین خستنەڕووی هۆکارەکانی سستی داوایکرد شەڕی مەرکەزی ناکۆکییەکان کەشەڕی کورد لەدەرەوەی شارەکانی کێشە بەخزمەتکردن خزمەتگوزاری ئەولەویەت بدات بەسنووری وەزارەتێکی کارەی نەکرد لەبەشەکەی متمانەی لێوەربگیرێتەوە شێخ جەعفەر مستەفاش وێڕای پڕۆژەیەک پاراستنی ئاسایشی دووپاتیکردەوە کەئەگەر پڕۆژەیە جێبەجێنەکرا پێشمەرگە شوێنێک لەو دەمێنێتەوەو هیچ ناکات پاشان بەتێکڕای دەنگ پەسەندکرا سکالاو گازندەکان پەسەندکردنی حاکم قادر حەمەجان عەزیزەوە کەچەندین خاڵی تێدا خرابووەڕوو هەم وەکو گازەندەو ڕەخنە ڕاسپاردە زیندووێتی ژیانی ڕێکخراوەیی خالانەدا پابەندبوون بەپەیڕەوی ناوخۆی نەهێشتنی ناعەدالەتی ڕێگەگرتن لەزیادەڕۆیی دەسەلاتی هەندێک لەلێپرسراوان دووپاتکرابووەوە پێشنیازی گرنگ واقیعیش خرانەڕوو کەهەمووی بەئاراستەی بەهێزکردن گوڕوتین بەخشین بوو ئاماژەش کرابوو بەوەی کەزیاتر بایەخ بەژنان پێگەی ژن گەنجان کەسوکاری شەهیدان ئەنفالکراو کەمئەندامانی سەنگەر بدرێت یاداشتکرا کەیاسای کادیران ڕێکبخرێت بنەماو ڕێساکان کاریش بەپاداشت سزا ڕوونکرابووەوە کەسنوورێک دەستوەردانی لێپرسراوان لەبازرگانی کۆکردنەوەی سامان دابنرێت بنبڕکردنی دەستەگەری دەربڕینی وردی موناقەشەی دڵسۆزانە ڕەخنەی بنیاتنەر دروستکەری بەهێز هەروەها بەرلەوەی لیژنەکە بخرێتە دەنگدانەوە کۆسرەت ڕەسوڵ داوای چەند دەقیقەیەک دەرفەتی قسەکردنی کردو ڕێگەپێدانی گوتەیەکی پێشکەشکرد تەکەتولی دووپاتکردەوە داواشیکرد بەزووترین کات نەزاهە پێکبهێنرێت زانینی سەروەت سامانی داواشی لەئەندامانی ڕیزەکانی ڕێکخستن کرد کەیەکێتیی بپارێزن بەناوی سەرکردایەتیشەوە بەڵێنیدا کەپابەندی گفتانەبن کەدراوەو دەدرێت گوتی جێبەجێنەکران لەکۆنگرەدا سەرکردایەتییە هەڵمەبژێرن خرایە دەنگی پەیامی لەڕاگەیاندنی بڕیاری پێکهێنانی ڕاگەیاندن پێکهات لەوادەی دیاریکراوی خۆیدا کۆ بووەوە لەسێیەم ڕۆژی پلینۆمدا ئازاد جندیانی ڕاپۆرتەکەی خوێندەوە کۆنفرانسەی ڕاگەیاندنی لەناوەڕاستی مانگی ئۆکتۆبەردا بەسترا پێشخستن کرانەوەی تێبینییەکانی لیژنەکەی نەبوونی پەیام کەپەرشوبلاوی لەڕاگەیاندنەکاندا دروستکردووە کورتخایەن درێژخایەن گەشەپێدانی لەوبارەیەوە ڕوونکراوانەی لەنێو ڕاپۆرتەکەدا لەلیژنەکەدا پێشکەشیکردبوو ڕوونیکردەوە بەرجەستەیە پەیامیشی گەڕاندەوە لەمەکتەبی سیاسیدا درێژخایەنیش پلانە لەهەموو مەکتەب ئۆرگانەکانی کورستاندا پەیڕەوبکرێت ڕاو بۆچوونی تێروتەسەل بەزۆرینەی بایەخدانی بەکەسوکاری لەدانیشتنی دوانیوەڕۆی سێیەم لیژنەیەکی پەسەندکران لەدانیشتنەکەدا بۆردومانی کیمیایی ئەنفال وارسەکانیان سەنگەرو زیندانیانی پێشکەشکران عەبدولکەریم هەڵەدنی خوێندیەوە پڕۆژە لەخۆگرتبوو لەبەرژەوەندیی کەیسانە گرنگیدان بەوارسی قوربانیانی چەکی تێکڕای خانووبەرە خوێندنی کوڕان کچانی شەهید وارسی تاچڕکردنەوەی هەوڵەکان کۆمەڵکوژی لەجیهاندا بەجینۆساید بناسرێت دروستکردنی مۆنیومێنت هێنانەوەی ڕوفاتی ئەنفالکراوەکان پێشنیازو لەڕاپۆرتی لیژنەکەدا قەرەبووبکرێنەوەو ژیانێکی شایستەیان دابینبکرێت کەبارتەقای قوربانیەکانیان تەئکیدکرایەوە کەدەبێت تێکۆشەرانی کۆچکردوو پۆلێنبکرێن تاوەکو ئەوانیش ژیانیان پڕۆژەکە لەبارەوەکراو بەئازادی ڕاشکاوی تێبینییەکانیان داکۆکییان لەمافی ڕەخنەش کەموکوڕیانە گیرا کەدەرهەق بەلایەنە پەیوەستەکانی کەیسەوە لەکۆتاییشدا مەکتەبەکانی لەڕاپۆرتەکەدا ئاماژە کرا ڕێکخراوە دێموکراتییەکان کەهەوڵبدات یەکخستنەوەی ڕێکخراوو کۆمەڵەکان گوێدانە ئینتیمای هاوئاهەنگیی نێوان داوایەکی لیژنەیەبوو لەکەموکورتییەکان داوایکردبوو کارئاسانی چاودێری ئۆرگانەکان بەسیستمی دامەزراوەیی لەکەموکورتی لەمەکتەبەکان مەکتەبەکان پەیڕەوی ناوخۆیان'
list_words = string_word.split()

model = load_model('nextword2.h5')
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

for word in list_words:
    
    try:
        seq = np.array(tokenizer.texts_to_sequences([word])[0])
        predd = model.predict_classes(seq)

        for key, value in tokenizer.word_index.items():
                    if value == predd:
                        predicted_word = key
                        break
        print('seq ->',str(tokenizer.index_word[int(seq)]))
        print('key ->',key)
        print('value ->',value)
        print('predicred ->', predicted_word)
        print('')
    except: continue
        
        

In [ ]:
# Importing the Libraries

from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer

model = load_model('nextword2.h5')
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
#         print(preds)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                
                break
        
        print(predicted_word)
        
        return predicted_word

In [ ]:
"""
    We are testing our model and we will run the model
    until the user decides to stop the script.
    While the script is running we try and check if 
    the prediction can be made on the text. If no
    prediction can be made we just continue.

"""

# text1 = "at the dull"
# text2 = "collection of textile"
# text3 = "what a strenuous"
# text4 = "stop the script"

while(True):

    text = input("Enter your line: ")
    
    if text == "stop":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-1]

            text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)
            
        except:
            continue

Enter your line: هەروا
Enter your line: stop
Ending The Program.....


In [ ]:
model2 = model.save('my_model2.h5', )

In [ ]:
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

In [ ]:
seq = np.array(tokenizer.texts_to_sequences(['و'])[0])
predd = model.predict(seq)
predd

In [ ]:
for key, value in tokenizer.word_index.items():
            if (value == predd).all():
                predicted_word = key
                break
        
print(predicted_word)

In [ ]:
def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
#         print(preds)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                break
        
        print(predicted_word)
        return predicted_word

In [ ]:
"""
    We are testing our model and we will run the model
    until the user decides to stop the script.
    While the script is running we try and check if 
    the prediction can be made on the text. If no
    prediction can be made we just continue.

"""

# text1 = "at the dull"
# text2 = "collection of textile"
# text3 = "what a strenuous"
# text4 = "stop the script"

while(True):

    text = input("Enter your line: ")
    
    if text == "stop the script":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-1]

            text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)
            
        except:
            continue